In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
import requests 
import pandas as pd 
import time
import re
import string
import nltk
import numpy as np
from nltk.corpus import stopwords

nltk.download('stopwords')
pd.set_option('display.max_colwidth', None)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
def isnotNaN(num):          #return true only if num is not NaN
    return num == num 

def extract_text(df):     # extract the text from the tweets, works only with .pkl file

  list_strings = []
  for index in range(len(df)):

      #print(index)
      if isnotNaN(df.loc[index]['retweeted_status']):             #if it is a RT I take the text from the original tweet
        if (df.loc[index]['retweeted_status']['truncated']) == True :                   #if the text is truncated I take the full text
          string = (df.loc[index]['retweeted_status']['extended_tweet'])["full_text"]
        else : 
          string = df.loc[index]['retweeted_status']['text']                            #if the text is not truncated I take simply the text
      elif df.loc[index]['truncated'] == True :                   #if the text is truncated and it is not a RT I take the full text
        string = (df.loc[index]['extended_tweet'])["full_text"]
      else : 
        string = df.loc[index]['text']                            #if the text is not truncated and it is not a RT I take simply the text
        
      #print(string)   
      list_strings.append(string)


  return list_strings


def who_clean(df):     #TO RUN ONLY on tweets collected using "WHO" keyword
                       #df is actually a list of strings

  list_strings = []
  for index in range(len(df)):
    string = df[index]
    if string.find("WHO") != -1:
       list_strings.append(string)    #only if the tweet contains "WHO" I add it to the list

  return list_strings


In [102]:
def cleaner(tweet,word_list = stopwords.words('english')):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    tweet = tweet.translate(table)# remove punctuation  

    special_characters = ['rt','dr','“','rt','the','to','of','a','an','on','has','have','had','is','was','did','n','et','ha','lt','1','2','5']

    tweet = tweet.split()      
    tweet = ' '.join([ word for word in tweet if word not in word_list and word not in special_characters  ])   #remove useless words
        
    return tweet

In [103]:
tweets_ap = pd.read_pickle("/content/drive/MyDrive/NS data project/AP/AP_WHO_202103170000_202104170000.pkl")
df_copy = tweets_ap.copy()
#tweets_filtered["clean_text"] = tweets_filtered["text"].map(cleaner)
#column_list = ["text","clean_text"]
#tweets_filtered = tweets_filtered[column_list]
#tweets_filtered
#display(tweets_filtered["clean_text"])
#print(tweets_filtered["clean_text"].to_string())


  

In [107]:
df_text = extract_text(df_copy)
df_text[0:10]
#cleaned_df_text = [cleaner(el) for el in df_text]
#cleaned_df_text[0:10]

['Prosecutors who oversaw a grand jury investigation into the death of Daniel Prude last year in Rochester, New York, undercut the case with testimony from a medical expert who said officers who held Prude down until he stopped breathing did nothing wrong. https://t.co/Xtwj1WqQ7U',
 'New Englanders who had enjoyed an early spring were brought back to a cold, blustery reality with a storm packing powerful wind gusts, lashing rain and heavy, wet snow. A foot of snow had fallen in western New Hampshire by midday. https://t.co/cAFFBhWet2',
 'Actor Helen McCrory, who played a crime family matriarch in "Peaky Blinders" and the scheming Narcissa Malfoy in the "Harry Potter" films, has died at age 52. Her husband, actor Damian Lewis, says she died at home after a "heroic battle with cancer."\n\nhttps://t.co/ZiGZarUzzM',
 'Liberty University is suing former leader Jerry Falwell Jr. for millions in damages. It says Falwell failed to disclose an alleged extortion attempt by a man who had a yearsl

In [105]:
df_clean = who_clean(df_text)
df_clean

['The World Health Organization says the pandemic death toll has surpassed 1 million in WHO’s Europe region, which includes Russia and stretches into Central Asia   https://t.co/AKvPr9BYpK',
 "Virus update: \n• WHO: Europe surpasses 1 million virus deaths.\n• Homeless Americans get a chance at COVID-19 shot.\n• Indian cities close as new virus cases hit 200K.\n• Bangkok nightlife reveals Thailand's virus stumbles.\n\nFollow all @AP coverage: https://t.co/VXkYBHXIdr",
 'India says it is approving the use of all coronavirus vaccines that have been given an emergency nod by the WHO or regulators in the U.S., Europe, Britain or Japan. The country is seeing a crippling surge of infections that could overwhelm hospitals. https://t.co/7HxhWNa6Lw',
 "Prosecutors are investigating whether a World Health Organization official lied to them and had a WHO report on Italy's coronavirus response spiked to spare the Italian government embarrassment and liability.  https://t.co/98m4QQitWH",
 'North Kor